In [1]:
import pandas as pd
import hashlib

In [2]:
from controle_coherence.controle_coherence import EngineDPE

In [4]:
engine=EngineDPE()

In [14]:
missing

['0101C',
 '0109C',
 '0602C',
 '1102C',
 '1106F',
 '1202C',
 '1203C',
 '1408C',
 '1501C',
 '1609C',
 '2306C',
 '2510C',
 '2514C',
 '2805C',
 '2806C',
 '3303C',
 '3306C',
 '3513C',
 '3601C',
 '3707C',
 '3801C',
 '3908C',
 '4104C',
 '4309C',
 '4310C',
 '4314C',
 '4506C',
 '4701C',
 '5714C',
 '6213C',
 '6716C',
 '6821C',
 '6910C',
 '6917C',
 '6919C',
 '6922C',
 '7002C',
 '7005C',
 '7106C',
 '7109C',
 '7413C',
 '7725C',
 '7904C',
 '7905C',
 '8203C',
 '8811C',
 '8816C',
 '8903C',
 '9116C',
 '9202C',
 '9327C']

In [19]:
engine.valeur_table['reseau_chaleur_2022'].query('nouveau_reseau_2021_2022==1').identifiant_reseau.to_json(orient='records')

'["0102C","0111C","0403C","0508C","0604C","0606F","0817C","1412C","1706C","1908C","2212C","2213C","2512C","2709C","2906C","2907C","3115C","3324C","3424F","3425C","3426F","3515C","3712C","3713C","3819C","3823C","3907C","4248C","4249C","4250C","4251C","4252C","4409C","4914C","4915C","5006C","5108C","5109C","5304C","5305C","5415C","5504C","5720C","5922C","6314C","6601C","6735C","6736C","6737C","6738C","6739C","6812C","6813C","6932C","6933C","7206C","7320C","7321C","7428C","7475C","7476C","7723C","8305C","8306C","8307C","8307F","8404C","8405C","8705C","8817C","8818C","8820C","8821C","9234C","9428C","9519C"]'

In [11]:
aset(engine.valeur_table['reseau_chaleur_2021'].identifiant_reseau)-set(.identifiant_reseau)

{'0101C',
 '0109C',
 '0602C',
 '1102C',
 '1106F',
 '1202C',
 '1203C',
 '1408C',
 '1501C',
 '1609C',
 '2306C',
 '2510C',
 '2514C',
 '2805C',
 '2806C',
 '3303C',
 '3306C',
 '3513C',
 '3601C',
 '3707C',
 '3801C',
 '3908C',
 '4104C',
 '4309C',
 '4310C',
 '4314C',
 '4506C',
 '4701C',
 '5714C',
 '6213C',
 '6716C',
 '6821C',
 '6910C',
 '6917C',
 '6919C',
 '6922C',
 '7002C',
 '7005C',
 '7106C',
 '7109C',
 '7413C',
 '7725C',
 '7904C',
 '7905C',
 '8203C',
 '8811C',
 '8816C',
 '8903C',
 '9116C',
 '9202C',
 '9327C'}

In [2]:
df_new = pd.read_excel('concordance arrêté 2021.xlsx',sheet_name='new')

In [3]:
df_old = pd.read_excel('concordance arrêté 2021.xlsx',sheet_name='old')

In [4]:
df_old['id_correspondance']=range(0,len(df_old))

In [5]:
df_new['id_correspondance']=range(0,len(df_new))

In [6]:
df_tv = pd.read_excel(r'D:\Cstbdata\MONWORKSPACE\Gitlab\observatoire-dpe\modele_donnee\valeur_tables.xlsx',sheet_name='reseau_chaleur')

In [7]:
reseau_chaleur = df_old.dropna(subset=['nom_reseau'])

In [8]:

hash_=reseau_chaleur['departement'].astype(int).astype(str).str.zfill(2)+reseau_chaleur['nom_reseau']+reseau_chaleur['chaud_ou_froid']

hash_=hash_.apply(lambda x:hashlib.md5(x.lower().strip().encode()).hexdigest()[0:6])

reseau_chaleur['hash_reseau']=hash_

<ipython-input-8-ea5e87bfab88>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reseau_chaleur['hash_reseau']=hash_


In [9]:
m=df_tv.merge(reseau_chaleur,on='hash_reseau',how='outer',suffixes=('','_old'))

In [10]:
m.columns

Index(['tv_reseau_chaleur_id', 'departement', 'hash_reseau', 'nom_reseau',
       'localisation', 'chaud_ou_froid', 'contenu_co2', 'taux_enr',
       'est_vertueux', 'code', 'departement_old', 'nom_reseau_old',
       'localisation_old', 'chaud_ou_froid_old', 'contenu_co2_old',
       'id_correspondance'],
      dtype='object')

In [11]:
m_new=df_new.merge(m[['tv_reseau_chaleur_id','id_correspondance','hash_reseau']],on='id_correspondance',how='left')

In [12]:
m_new=m_new.dropna(subset=['identifiant_reseau'])

In [13]:
m_new.tv_reseau_chaleur_id = m_new.tv_reseau_chaleur_id.astype(pd.Int32Dtype())

In [16]:
m_new=m_new.rename(columns={'is_new':'nouveau_reseau','tv_reseau_chaleur_id':'tv_reseau_chaleur_id_old','hash_reseau':'hash_reseau_old'})

In [17]:
m_new.to_excel('reseau_chaleur_nouveau.xlsx')

In [128]:
null_and_old = (m_new.tv_reseau_chaleur_id.isnull()) &( m_new.is_new!=1)

In [133]:
df_tv.loc[df_tv.localisation.str.lower().str.contains('courneuve')]

,tv_reseau_chaleur_id,departement,hash_reseau,nom_reseau,localisation,chaud_ou_froid,contenu_co2,taux_enr,est_vertueux
754,755,93,8b6c3b,Quartier Nord,La Courneuve,C,0.088,NaN,NaN
760,761,93,d220b2,Quartier Sud,La Courneuve,C,0.170,NaN,NaN


In [21]:
m_new.identifiant_reseau.str.match(r'[0-9]{4}[C-F]').mean()

1.0

In [30]:
{1,12} & {2,4}

set()

In [28]:
{1,2,14} - {1,2,12} - {1,12}

{14}

In [134]:
m_new.loc[m_new.localisation.str.lower().str.contains('courneuve')]

,identifiant_reseau,nom_reseau,localisation,contenu_co2,contenu_co2_acv,taux_enr,methode_calcul_taux,is_new,id_correspondance,tv_reseau_chaleur_id,hash_reseau
842,9313C,Réseau de La Courneuve,LA COURNEUVE,0.082,0.09,0.591,2020,NaN,842,NaN,cc4abb


In [129]:
m_new.loc[null_and_old]

,identifiant_reseau,nom_reseau,localisation,contenu_co2,contenu_co2_acv,taux_enr,methode_calcul_taux,is_new,id_correspondance,tv_reseau_chaleur_id,hash_reseau
842,9313C,Réseau de La Courneuve,LA COURNEUVE,0.082,0.09,0.591,2020,NaN,842,NaN,cc4abb


In [68]:
df_tv.shape

(797, 9)

In [69]:
reseau_chaleur.shape

(661, 7)

In [66]:
is_null = m.nom_reseau.isnull()
m.loc[is_null]

,tv_reseau_chaleur_id,departement,hash_reseau,nom_reseau,localisation,chaud_ou_froid,contenu_co2,taux_enr,est_vertueux,code,departement_old,nom_reseau_old,localisation_old,chaud_ou_froid_old,contenu_co2_old
797,NaN,NaN,cc4abb,NaN,NaN,NaN,NaN,NaN,NaN,TV046_363,93.0,La Courneuve - Quartier Sud,La Courneuve,C,0.170
798,NaN,NaN,12c939,NaN,NaN,NaN,NaN,NaN,NaN,TV046_356,93.0,La Courneuve - Quartier Nord,La Courneuve,C,0.088


In [67]:
is_null = m.nom_reseau_old.isnull()
m.loc[is_null]

,tv_reseau_chaleur_id,departement,hash_reseau,nom_reseau,localisation,chaud_ou_froid,contenu_co2,taux_enr,est_vertueux,code,departement_old,nom_reseau_old,localisation_old,chaud_ou_froid_old,contenu_co2_old
14,15.0,3.0,864124,Réseau de Cerilly,Cerilly,C,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,18.0,3.0,f40461,Réseau Bellenaves,Bellenaves,C,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,21.0,3.0,a627f6,SDC Moulins Ville,Moulins,C,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,24.0,4.0,fa9290,Complexe sportif,Forcalquier,C,0.223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,26.0,4.0,43a615,Réseau communal de Castellane,Castellane,C,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,755.0,93.0,8b6c3b,Quartier Nord,La Courneuve,C,0.088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760,761.0,93.0,d220b2,Quartier Sud,La Courneuve,C,0.170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,762.0,93.0,ef96b3,"Réseau de Rosny-sous-Bois, Noisy-le-Sec, Montr...",Rosny-sous-Bois,C,0.063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,768.0,94.0,4f49b9,Réseau Geotelluence,Ivry-sur-Seine,C,0.133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
m.nom_reseau.isnull

SyntaxError: EOL while scanning string literal (<ipython-input-61-f324d53555b2>, line 1)

In [33]:
reseau_chaleur[['hash_reseau','hash_reseau_test']]

,hash_reseau,hash_reseau_test
0,e5edbe,e5edbe
1,f2cb1c,f2cb1c
2,5ed927,5ed927
3,73d0ee,73d0ee
4,ea85c1,ea85c1
...,...,...
792,a06130,a06130
793,2fe486,2fe486
794,6cd939,6cd939
795,113d5a,113d5a


In [22]:
from md5

SyntaxError: invalid syntax (<ipython-input-22-b52176df6f4c>, line 1)

In [5]:
df_tv

,tv_reseau_chaleur_id,departement,hash_reseau,nom_reseau,localisation,chaud_ou_froid,contenu_co2,taux_enr,est_vertueux
0,1,1,e5edbe,La Reyssouze,Bourg-en-Bresse,C,0.155,NaN,NaN
1,2,1,f2cb1c,Réseau d'Hauteville Lompnes,Hauteville Lompnes,C,0.128,NaN,NaN
2,3,1,5ed927,Oyonnax BioChaleur,Oyonnax,C,0.046,NaN,NaN
3,4,1,73d0ee,Réseau de Saint-Denis-les-Bourg,Saint-Denis-les-Bourg,C,0.045,NaN,NaN
4,5,1,ea85c1,Belena,Belley,C,0.029,NaN,NaN
...,...,...,...,...,...,...,...,...,...
792,793,95,a06130,ZUP de l'Epine Guyon,Franconville,C,0.178,NaN,NaN
793,794,95,2fe486,Réseau de Villiers-le-Bel-Gonesse,Villiers-le-Bel,C,0.083,NaN,NaN
794,795,95,6cd939,Réseau de Pontoise,Pontoise,C,0.209,NaN,NaN
795,796,95,113d5a,ZAC de Montedour,Franconville,C,0.229,NaN,NaN
